In [1]:
# Importing SparkSession 
from pyspark.sql import SparkSession

In [2]:
# Creating a SparkSession Object
spark = SparkSession. \
    builder. \
    config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1'). \
    config('spark.ui.port', '0'). \
    config('spark.sql.warehouse.dir', f'/user/warehouse'). \
    enableHiveSupport(). \
    appName('Python - Kafka and Spark Integration for Spark Streaming for CTA Project'). \
    master('yarn'). \
    getOrCreate()

:: loading settings :: url = jar:file:/opt/spark-3.5.0-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/guruprasadvk10/.ivy2/cache
The jars for the packages stored in: /home/guruprasadvk10/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bfa3f085-f433-468b-b420-42ea50bed11f;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.1 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 427ms :: artifacts dl 15ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.4-3 from central in [default]
	org.apache.commons#commons-pool2;2.

In [3]:
# Configuring the Bootstrap servers
kafka_bootstrap_servers = 'localhost:9092'

In [4]:
# Creating an object for ReadStream
df_cta = spark. \
  readStream. \
  format('kafka'). \
  option('kafka.bootstrap.servers', kafka_bootstrap_servers). \
  option('subscribe', 'cta_topic_kc'). \
  load()

In [5]:
# Validating if the Stream is active
df_cta.isStreaming

True

In [6]:
# Printing the schema of the Stream dataframe
df_cta.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [7]:
# Using console mode to create a Write Stream object to write to the stream every 30 seconds.
df_cta.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)"). \
    writeStream. \
    outputMode("update"). \
    format("console"). \
    option('truncate', 'false'). \
    trigger(processingTime='30 seconds'). \
    start()

24/02/26 00:49:43 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-48d0cca2-a272-424f-8f2c-ddf52681d7cb. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/02/26 00:49:43 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [8]:
# Using Format mode to create a Write Stream object to write to the stream. Here query name object df_cta_sql is also created
df_cta.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)"). \
    writeStream. \
    format("memory"). \
    queryName("df_cta_sql"). \
    start()

-------------------------------------------
Batch: 0
-------------------------------------------


24/02/26 00:49:46 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-dd3cf8d8-7e94-443f-b761-58715c136db3. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/02/26 00:49:46 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


+---+-----+
|key|value|
+---+-----+
+---+-----+



In [9]:
# Selecting count from the query name object
spark.sql('SELECT count(1) FROM df_cta_sql').show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



-------------------------------------------
Batch: 1
-------------------------------------------
+----+----------------------------------------------------------------------------------------------------------+
|key |value                                                                                                     |
+----+----------------------------------------------------------------------------------------------------------+
|NULL|pink,308,30114,Loop,40170,Ashland,2024-02-25T18:49:32,2024-02-25T18:51:32,0,41.88531,-87.66697            |
|NULL|pink,310,30114,54th/Cermak,40680,Adams/Wabash,2024-02-25T18:49:06,2024-02-25T18:51:06,0,41.88574,-87.62758|
|NULL|pink,313,30114,54th/Cermak,41030,Polk,2024-02-25T18:49:30,2024-02-25T18:51:30,0,41.88487,-87.67007        |
|NULL|pink,314,30114,Loop,40600,Kostner,2024-02-25T18:49:37,2024-02-25T18:51:37,0,41.85192,-87.74534            |
+----+---------------------------------------------------------------------------------------------------

In [10]:
# Selecting data from the query name object
spark.sql('SELECT * FROM df_cta_sql').show(truncate=False)

+----+----------------------------------------------------------------------------------------------------------+
|key |value                                                                                                     |
+----+----------------------------------------------------------------------------------------------------------+
|NULL|pink,308,30114,Loop,40170,Ashland,2024-02-25T18:49:32,2024-02-25T18:51:32,0,41.88531,-87.66697            |
|NULL|pink,310,30114,54th/Cermak,40680,Adams/Wabash,2024-02-25T18:49:06,2024-02-25T18:51:06,0,41.88574,-87.62758|
|NULL|pink,313,30114,54th/Cermak,41030,Polk,2024-02-25T18:49:30,2024-02-25T18:51:30,0,41.88487,-87.67007        |
|NULL|pink,314,30114,Loop,40600,Kostner,2024-02-25T18:49:37,2024-02-25T18:51:37,0,41.85192,-87.74534            |
|NULL|g,008,30004,Harlem/Lake,40510,Garfield,2024-02-25T18:49:31,2024-02-25T18:53:31,0,41.78605,-87.63203       |
|NULL|g,010,30139,Cottage Grove,41400,Roosevelt,2024-02-25T18:49:57,2024-02-25T18:51:57,

In [11]:
# Importing sql functions for processing
from pyspark.sql.functions import lit, date_format, to_date, split, substring,unix_timestamp, from_unixtime

In [12]:
# Casting the columns as String and Adding new columns to extract Year, month and date information
df_cta.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)"). \
    withColumn('route_color', split('value', ',')[0]). \
    withColumn('transit_date', to_date(split('value', ',')[6], "yyyy-MM-dd'T'HH:mm:ss")). \
    withColumn('year', date_format('transit_date', 'yyyy')). \
    withColumn('month', date_format('transit_date', 'MM')). \
    withColumn('dayofmonth', date_format('transit_date', 'dd')). \
    writeStream. \
    format("memory"). \
    queryName("df_cta_sql1"). \
    start()

24/02/26 00:50:12 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-6cfa1cc1-4a47-4b99-b005-b3da333857e0. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/02/26 00:50:12 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [13]:
# Selecting sample rows 
spark.sql('SELECT * FROM df_cta_sql1').show(truncate=False)

+----+-------------------------------------------------------------------------------------------------+-----------+------------+----+-----+----------+
|key |value                                                                                            |route_color|transit_date|year|month|dayofmonth|
+----+-------------------------------------------------------------------------------------------------+-----------+------------+----+-----+----------+
|NULL|brn,409,30249,Loop,41480,Western,2024-02-25T18:49:46,2024-02-25T18:50:46,0,41.96624,-87.69073    |brn        |2024-02-25  |2024|02   |25        |
|NULL|brn,410,30249,Loop,40530,Diversey,2024-02-25T18:49:41,2024-02-25T18:51:41,0,41.93975,-87.65338   |brn        |2024-02-25  |2024|02   |25        |
|NULL|brn,413,30249,Kimball,40710,Chicago,2024-02-25T18:49:58,2024-02-25T18:50:58,0,41.89678,-87.63595 |brn        |2024-02-25  |2024|02   |25        |
|NULL|brn,414,30249,Kimball,40040,Quincy,2024-02-25T18:49:55,2024-02-25T18:50:55,0,41.88

In [14]:
!hdfs dfs -rm -R -skipTrash /final_project_hdfs

Deleted /final_project_hdfs
-------------------------------------------
Batch: 2
-------------------------------------------
+----+------------------------------------------------------------------------------------------------------+
|key |value                                                                                                 |
+----+------------------------------------------------------------------------------------------------------+
|NULL|g,008,30004,Harlem/Lake,40510,Garfield,2024-02-25T18:49:31,2024-02-25T18:53:31,0,41.78605,-87.63203   |
|NULL|g,010,30139,Cottage Grove,41400,Roosevelt,2024-02-25T18:49:57,2024-02-25T18:51:57,0,41.87467,-87.62643|
|NULL|g,013,30004,Harlem/Lake,41160,Clinton,2024-02-25T18:49:26,2024-02-25T18:51:26,0,41.88574,-87.63089    |
|NULL|g,601,30004,Harlem/Lake,41400,Roosevelt,2024-02-25T18:49:56,2024-02-25T18:50:56,0,41.86436,-87.62654  |
|NULL|g,609,30057,Ashland/63rd,41080,47th,2024-02-25T18:48:36,2024-02-25T18:49:36,0,41.81284,-87.61892   

### Writing Spark Streaming data to HDFS

In [15]:
# Writing to HDFS and partitioning the data using Year, Month and Day columns. Data is written to HDFS every 30 seconds from the Spark stream.
df_cta.selectExpr("CAST(value AS STRING)"). \
    withColumn('route_color', split('value', ',')[0]). \
    withColumn('transit_date', to_date(split('value', ',')[6], "yyyy-MM-dd'T'HH:mm:ss")). \
    withColumn('year', date_format('transit_date', 'yyyy')). \
    withColumn('month', date_format('transit_date', 'MM')). \
    withColumn('dayofmonth', date_format('transit_date', 'dd')). \
    writeStream. \
    partitionBy('year', 'month', 'dayofmonth'). \
    format('csv'). \
    option("checkpointLocation", '/final_project_hdfs/checkpoint'). \
    option('path', '/final_project_hdfs/data'). \
    option('header',True). \
    option('sep','|'). \
    trigger(processingTime='30 seconds'). \
    start()

24/02/26 00:50:36 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [16]:
# Validating the HDFS directories
!hdfs dfs -ls /final_project_hdfs

Found 2 items
drwxr-xr-x   - guruprasadvk10 supergroup          0 2024-02-26 00:50 /final_project_hdfs/checkpoint
drwxr-xr-x   - guruprasadvk10 supergroup          0 2024-02-26 00:50 /final_project_hdfs/data


In [19]:
!hdfs dfs -ls -R /final_project_hdfs/data/year=2024

drwxr-xr-x   - guruprasadvk10 supergroup          0 2024-02-26 00:51 /final_project_hdfs/data/year=2024/month=02
drwxr-xr-x   - guruprasadvk10 supergroup          0 2024-02-26 00:51 /final_project_hdfs/data/year=2024/month=02/dayofmonth=25
-rw-r--r--   1 guruprasadvk10 supergroup       1891 2024-02-26 00:51 /final_project_hdfs/data/year=2024/month=02/dayofmonth=25/part-00000-559278f9-7ecf-4adc-9990-1b132b4bb10f.c000.csv


In [20]:
!hdfs dfs -ls /final_project_hdfs/checkpoint

Found 4 items
drwxr-xr-x   - guruprasadvk10 supergroup          0 2024-02-26 00:51 /final_project_hdfs/checkpoint/commits
-rw-r--r--   1 guruprasadvk10 supergroup         45 2024-02-26 00:50 /final_project_hdfs/checkpoint/metadata
drwxr-xr-x   - guruprasadvk10 supergroup          0 2024-02-26 00:51 /final_project_hdfs/checkpoint/offsets
drwxr-xr-x   - guruprasadvk10 supergroup          0 2024-02-26 00:50 /final_project_hdfs/checkpoint/sources


In [21]:
!hdfs dfs -cat /final_project_hdfs/checkpoint/sources/0/0

 v1
{"cta_topic_kc":{"0":204}}

In [22]:
!hdfs dfs -ls /final_project_hdfs/checkpoint/offsets

Found 2 items
-rw-r--r--   1 guruprasadvk10 supergroup        667 2024-02-26 00:50 /final_project_hdfs/checkpoint/offsets/0
-rw-r--r--   1 guruprasadvk10 supergroup        667 2024-02-26 00:51 /final_project_hdfs/checkpoint/offsets/1


In [23]:
!hdfs dfs -cat /final_project_hdfs/checkpoint/offsets/0

v1
{"batchWatermarkMs":0,"batchTimestampMs":1708908636653,"conf":{"spark.sql.streaming.stateStore.providerClass":"org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider","spark.sql.streaming.join.stateFormatVersion":"2","spark.sql.streaming.stateStore.compression.codec":"lz4","spark.sql.streaming.stateStore.rocksdb.formatVersion":"5","spark.sql.streaming.statefulOperator.useStrictDistribution":"true","spark.sql.streaming.flatMapGroupsWithState.stateFormatVersion":"2","spark.sql.streaming.multipleWatermarkPolicy":"min","spark.sql.streaming.aggregation.stateFormatVersion":"2","spark.sql.shuffle.partitions":"200"}}
{"cta_topic_kc":{"0":204}}-------------------------------------------
Batch: 4
-------------------------------------------
+----+--------------------------------------------------------------------------------------------------------+
|key |value                                                                                                   |
+----+------

In [24]:
# Validating the data written in HDFS by creating spark dataframe
df_from_hdfs1=spark.read.csv("/final_project_hdfs/data/",sep="|",header=True)

In [25]:
# Checking schema
df_from_hdfs1.printSchema()

root
 |-- value: string (nullable = true)
 |-- route_color: string (nullable = true)
 |-- transit_date: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- dayofmonth: integer (nullable = true)



In [26]:
# CHecking sample rows
df_from_hdfs1.show(truncate=False)

+--------------------------------------------------------------------------------------------------------+-----------+------------+----+-----+----------+
|value                                                                                                   |route_color|transit_date|year|month|dayofmonth|
+--------------------------------------------------------------------------------------------------------+-----------+------------+----+-----+----------+
|org,708,30182,Midway,41400,Roosevelt,2024-02-25T18:50:24,2024-02-25T18:53:24,0,41.8795,-87.6261         |org        |2024-02-25  |2024|2    |25        |
|org,710,30182,Loop,41130,Halsted,2024-02-25T18:50:28,2024-02-25T18:52:28,0,41.83957,-87.66485           |org        |2024-02-25  |2024|2    |25        |
|org,711,30182,Midway,40930,Midway,2024-02-25T18:50:37,2024-02-25T18:51:37,0,41.79572,-87.735            |org        |2024-02-25  |2024|2    |25        |
|org,712,30182,Midway,40310,Western,2024-02-25T18:50:52,2024-02-25T18:51:52,

### Creating tables in HIVE and querying the tables

In [27]:
# Checking list of databases in Hive
spark.sql("show databases")

24/02/26 00:51:44 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/02/26 00:51:44 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


namespace
cta_db
default
retail_db


In [28]:
# Using the cta_db created for this project
spark.sql("use cta_db")

24/02/26 00:51:48 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


""


In [29]:
# Drop the table cta_data is it exists already
spark.sql("DROP TABLE IF EXISTS cta_data;")

""


In [30]:
# Creating Hive table structure
spark.sql("CREATE TABLE cta_data (    value STRING,    route_color  STRING,    transit_date STRING) ROW FORMAT DELIMITED FIELDS TERMINATED BY '|';")


24/02/26 00:51:54 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
24/02/26 00:51:54 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
24/02/26 00:51:54 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/02/26 00:51:54 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/02/26 00:51:54 WARN HiveMetaStore: Location: hdfs://localhost:9000/user/hive/warehouse/cta_db.db/cta_data specified for non-external table:cta_data


""


In [31]:
# Loading data into Hive table using HDFS data
spark.sql("LOAD DATA INPATH '/final_project_hdfs/data/year=2024/month=02/dayofmonth=25/' INTO TABLE cta_data;")

""


-------------------------------------------
Batch: 5
-------------------------------------------
+----+----------------------------------------------------------------------------------------------------------+
|key |value                                                                                                     |
+----+----------------------------------------------------------------------------------------------------------+
|NULL|brn,409,30249,Loop,40090,Damen,2024-02-25T18:51:01,2024-02-25T18:52:01,0,41.96634,-87.68351               |
|NULL|brn,410,30249,Loop,40530,Diversey,2024-02-25T18:51:00,2024-02-25T18:52:00,0,41.93602,-87.65327            |
|NULL|brn,413,30249,Kimball,40800,Sedgwick,2024-02-25T18:50:51,2024-02-25T18:53:51,0,41.89678,-87.63595         |
|NULL|brn,414,30249,Kimball,40160,LaSalle/Van Buren,2024-02-25T18:51:00,2024-02-25T18:53:00,0,41.87872,-87.63374|
|NULL|brn,415,30249,Kimball,40090,Damen,2024-02-25T18:50:24,2024-02-25T18:52:24,0,41.96175,-87.67517     

In [32]:
# Selecting sample rows from cta_data tables
spark.sql("SELECT * FROM cta_data LIMIT 5;")

value,route_color,transit_date
value,route_color,transit_date
"org,708,30182,Mid...",org,2024-02-25
"org,710,30182,Loo...",org,2024-02-25
"org,711,30182,Mid...",org,2024-02-25
"org,712,30182,Mid...",org,2024-02-25


In [33]:
# Selecting count of rows from cta_data tables
spark.sql("SELECT count(1) FROM cta_data;")

count(1)
38


In [34]:
spark.sql("SELECT * FROM cta_data where route_color='g';")

value,route_color,transit_date
"g,008,30004,Harle...",g,2024-02-25
"g,010,30139,Cotta...",g,2024-02-25
"g,013,30004,Harle...",g,2024-02-25
"g,601,30004,Harle...",g,2024-02-25
"g,609,30057,Ashla...",g,2024-02-25
"g,610,30004,Harle...",g,2024-02-25
"g,613,30057,Ashla...",g,2024-02-25
"g,615,30139,Cotta...",g,2024-02-25
